In [1]:
import os
import mne
import numpy as np
import scipy.io

def load_ica_data(subject_code):
    ica_file = f'DATA/EEG_ICA_STRUCT/PTW{subject_code}_allwalk_EEG_ICA_STRUCT_rejbadchannels_diverse_incr_comps.mat'
    ica_data = scipy.io.loadmat(ica_file)
    icaweights = ica_data['EEG_ICA_STRUCT']['weights'][0][0]
    icasphere = ica_data['EEG_ICA_STRUCT']['sphere'][0][0]
    good_channels = ica_data['EEG_ICA_STRUCT']['good_chans'][0][0][0] - 1  # Subtract 1 to correct the indices
    return icaweights, icasphere, good_channels

# def epoch_ica_data(ica_data, events, event_id, tmin=-0.05, tmax=0.05):
#     info = mne.create_info(ch_names=[f'IC{i+1}' for i in range(ica_data.shape[0])], sfreq=raw.info['sfreq'], ch_types='eeg')
#     ica_raw = mne.io.RawArray(ica_data, info)
#     epochs = mne.Epochs(ica_raw, events, event_id, tmin=tmin, tmax=tmax)
#     return epochs

def epoch_ica_data(ica_data, events, event_id, tmin=-0.1, tmax=0.1):
    info = mne.create_info(ch_names=[f'{i+1}' for i in range(ica_data.shape[0])], sfreq=raw.info['sfreq'], ch_types='eeg')
    ica_raw = mne.io.RawArray(ica_data, info)
    epochs = mne.Epochs(ica_raw, events, event_id, tmin=tmin, tmax=tmax)
    return epochs


def apply_ica_weights(raw, icaweights, icasphereou, good_channels, num_ics=78):
    # Get the channel names for the good channels
    good_channel_names = [raw.ch_names[i] for i in good_channels]

    # Get the data from the good channels only
    good_channel_data = raw.copy().pick_channels(good_channel_names).get_data()

    # Compute the unmixing matrix using the provided ICA weights and sphere matrix
    unmixing_matrix = np.dot(icaweights, icasphere)

    # Select the top 78 components based on the sum of the absolute weights
    component_weights = np.sum(np.abs(unmixing_matrix), axis=1)
    top_ics_indices = np.argsort(component_weights)[-num_ics:]

    # Apply the unmixing matrix for the top 78 ICA components to the good channel data
    ica_data = np.dot(unmixing_matrix[top_ics_indices, :], good_channel_data)

    return ica_data


subject_ids = range(1, 15)
event_id = {'ULHS': 11, 'ULTO': 13, 'URHS': 14, 'URTO': 15}
save_dir = 'DATA/Event_Epoched_Data_ICA_78'  # Specify your desired save directory

# Make sure the save directory exists
os.makedirs(save_dir, exist_ok=True)

all_epochs = {event: [] for event in event_id}

for subject in subject_ids:
    subject_code = f'{subject:02d}'
    set_file = f'DATA/Data_Inner_Layer/PTW{subject_code}_allwalk_EEG.set'

    # Load the .set file
    raw = mne.io.read_raw_eeglab(set_file, preload=True)

    # Load the ICA data
    icaweights, icasphere, good_channels = load_ica_data(subject_code)

    # Apply the ICA weights to the raw data using the provided ICA weights and sphere matrix
    ica_data = apply_ica_weights(raw, icaweights, icasphere, good_channels)

    # Get the events from the raw data
    events, event_id_list = mne.events_from_annotations(raw)

    # Epoch the ICA activations around the events
    epochs = epoch_ica_data(ica_data, events, event_id)

    for event in event_id:
        all_epochs[event].append(epochs[event])

# Concatenate and save the epoched data for each event type
for event in event_id:
    concatenated_epochs = mne.concatenate_epochs(all_epochs[event])
    save_path = f'{save_dir}/my_epochs_{event}-epo.fif'
    concatenated_epochs.save(save_path, overwrite=True)

# Print the count of events for each event type
for event in event_id:
    event_count = sum([len(epochs[event]) for epochs in all_epochs[event]])
    print(f'Total number of events for {event}: {event_count}')


Reading c:\Users\kanka\OneDrive\Desktop\Predicting_Gait_Events_Using_Dual_layer_EEG_dataset\DATA\Data_Inner_Layer\PTW01_allwalk_EEG.fdt
Reading 0 ... 1536511  =      0.000 ...  3000.998 secs...


C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:61: RuntimeWarning: Estimated head radius (11.3 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(set_file, preload=True)
C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:61: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_file, preload=True)


Used Annotations descriptions: ['CLHS', 'CLP_On', 'CLTO', 'CRHS', 'CRTO', 'PLHS', 'PLP_On', 'PLTO', 'PRHS', 'PRTO', 'ULHS', 'ULP_On', 'ULTO', 'URHS', 'URTO', 'boundary']
Creating RawArray with float64 data, n_channels=78, n_times=1536512
    Range : 0 ... 1536511 =      0.000 ...  3000.998 secs
Ready.
Not setting metadata
3244 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Reading c:\Users\kanka\OneDrive\Desktop\Predicting_Gait_Events_Using_Dual_layer_EEG_dataset\DATA\Data_Inner_Layer\PTW02_allwalk_EEG.fdt
Reading 0 ... 1543167  =      0.000 ...  3013.998 secs...


C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:61: RuntimeWarning: Estimated head radius (11.7 cm) is above the 99th percentile for adult head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(set_file, preload=True)
C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:61: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(set_file, preload=True)


Used Annotations descriptions: ['CLHS', 'CLP_On', 'CLTO', 'CRHS', 'CRTO', 'PLHS', 'PLP_On', 'PLTO', 'PRHS', 'PRTO', 'ULHS', 'ULP_On', 'ULTO', 'URHS', 'URTO', 'boundary']
Creating RawArray with float64 data, n_channels=78, n_times=1543168
    Range : 0 ... 1543167 =      0.000 ...  3013.998 secs
Ready.
Not setting metadata
3224 matching events found
Setting baseline interval to [-0.099609375, 0.0] sec
Applying baseline correction (mode: mean)
0 projection items activated
Reading c:\Users\kanka\OneDrive\Desktop\Predicting_Gait_Events_Using_Dual_layer_EEG_dataset\DATA\Data_Inner_Layer\PTW03_allwalk_EEG.fdt
Reading 0 ... 1546239  =      0.000 ...  3019.998 secs...


...

Using data from preloaded Raw for 811 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 806 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 831 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 769 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 804 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 781 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 784 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 816 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 788 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 767 events and 103 original time points ...
0 bad epochs dropped
Using data

C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:82: RuntimeWarning: epochs.drop_log contains 170222 entries which will incur up to a 2.1 MB writing overhead (per split file), consider using epochs.reset_drop_log_selection() prior to writing
  concatenated_epochs.save(save_path, overwrite=True)


Using data from preloaded Raw for 811 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 806 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 831 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 769 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 804 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 781 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 784 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 816 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 788 events and 103 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 767 events and 103 original time points ...
0 bad epochs dropped
Using data

C:\Users\kanka\AppData\Local\Temp\ipykernel_27708\3270261566.py:82: RuntimeWarning: epochs.drop_log contains 170222 entries which will incur up to a 2.1 MB writing overhead (per split file), consider using epochs.reset_drop_log_selection() prior to writing
  concatenated_epochs.save(save_path, overwrite=True)


Total number of events for ULHS: 11034
Total number of events for ULTO: 11034
Total number of events for URHS: 11034
Total number of events for URTO: 11034
